In [1]:
# pip install folium
# pip install geopy
# !pip install utm --user

In [2]:
import os
import utm
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
from geopy.distance import geodesic

import warnings
warnings.filterwarnings("ignore")
%config InlineBackend.figure_format = 'retina'
cols = ['uid','total_distance', 'average_speed', 'max_elevation', 'min_elevation', 'average_elevation']

C:\Users\avitr\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
os.listdir()

['.ipynb_checkpoints',
 'doc1 (1).docx',
 'gps_movement_map.html',
 'trackdata_c1.csv',
 'trackdata_c2.csv',
 'trackdata_cx.csv',
 'Untitled.ipynb']

In [4]:
df1 = pd.read_csv('trackdata_c1.csv')
df2 = pd.read_csv('trackdata_c2.csv')
dfx = pd.read_csv('trackdata_cx.csv')

In [5]:
def get_features(train):
    train['longitude_diff'] = train.groupby('uid').lon.diff().fillna(0)
    train['latitude_diff'] = train.groupby('uid').lat.diff().fillna(0)
    train['elevation_diff'] = train.groupby('uid').ele.diff().fillna(0)

    train['UTMX'] = train.progress_apply(lambda x:utm.from_latlon(x['lat'], x['lon'])[0],axis=1)
    train['UTMY'] = train.progress_apply(lambda x:utm.from_latlon(x['lat'], x['lon'])[1],axis=1)

    # Function to calculate distance between two points
    distance = lambda x_dif, y_dif: np.sqrt(x_dif**2 + y_dif**2)

    # Differencing UTM coordinates
    train['UTMX_diff'] = train.groupby('uid').UTMX.diff().fillna(0)
    train['UTMY_diff'] = train.groupby('uid').UTMY.diff().fillna(0)


    # Calculate step distance
    train['distance'] = distance(train.UTMX_diff, train.UTMY_diff)

    # Interquartile and range function
    iqr = lambda x: np.percentile(x, 75) - np.percentile(x, 25)
    range = lambda x: np.max(x) - np.min(x)
    
    # Drop the last 1 column: label
    
    if 'Target' and 'row_number' in train:
        df_grouped = train.drop(['Target','row_number'],axis=1)
    else:
        df_grouped = train.drop(['row_number'],axis=1)

    # Calculate summary statistics
    df_grouped = df_grouped.groupby('uid').aggregate([np.mean, np.min, np.max, np.std, iqr, range])

    # Reduce multi-index
    df_grouped.columns = ['_'.join(col).strip() for col in df_grouped.columns.values]

    # Replace column name <lambda_0> to IQR and <lambda_1> to range 
    col_groupby2 = df_grouped.columns
    col_groupby2 = [w.replace('<lambda_0>', 'IQR') for w in col_groupby2]
    col_groupby2 = [w.replace('<lambda_1>', 'range') for w in col_groupby2]

    # Update names of columns
    df_grouped.columns = col_groupby2
    return df_grouped

In [6]:
df1['Target'] = 1 # fake
df2['Target'] = 0 # real
train_data = pd.concat([df1,df2])
train_data.reset_index(drop=True,inplace=True)

In [ ]:
train = get_features(train_data)

  0%|          | 0/3305525 [00:00<?, ?it/s]

In [ ]:
# def get_features(df_temp):

#     uid = df_temp['uid'].unique()[0]
#     # Calculate total distance traveled
#     total_distance = 0.0
#     for i in range(1, len(df_temp)):
#         coord1 = (df_temp['lat'][i-1], df_temp['lon'][i-1])
#         coord2 = (df_temp['lat'][i], df_temp['lon'][i])
#         total_distance += geodesic(coord1, coord2).kilometers

#     # Calculate average speed
#     total_time_hours = (df_temp['row_number'].iloc[-1] - df_temp['row_number'].iloc[0]) / 3600.0
#     average_speed = total_distance / total_time_hours

#     # Maximum elevation
#     max_elevation = df_temp['ele'].max()

#     # Minimum elevation
#     min_elevation = df_temp['ele'].min()

#     # Average elevation
#     average_elevation = df_temp['ele'].mean()

#     return uid,total_distance, average_speed, max_elevation, min_elevation, average_elevation

In [ ]:
# features = []
# for uid in tqdm(set(df1['uid'])):
#     temp = df1[df1['uid']==uid]
#     temp.reset_index(drop=True,inplace=True)
#     features.append(get_features(temp))

In [ ]:
# df1_cleaned = pd.DataFrame(features,columns=cols)

In [ ]:
# features = []
# for uid in tqdm(set(df2['uid'])):
#     temp = df2[df2['uid']==uid]
#     temp.reset_index(drop=True,inplace=True)
#     features.append(get_features(temp))

In [ ]:
# df2_cleaned = pd.DataFrame(features,columns=cols)

In [ ]:
# features = []
# for uid in tqdm(set(dfx['uid'])):
#     temp = dfx[dfx['uid']==uid]
#     temp.reset_index(drop=True,inplace=True)
#     features.append(get_features(temp))
# dfx_cleaned = pd.DataFrame(features,columns=cols)

In [ ]:
# df1_cleaned['Target'] = 1 # fake/issue
# df2_cleaned['Target'] = 0 # no issue

In [ ]:
# df_train = pd.concat([df1_cleaned,df2_cleaned])

In [ ]:
df_train = df_train.sample(frac=1).reset_index(drop=True)

In [ ]:
df_train['Target'].value_counts()

In [ ]:
X = df_train.drop(['uid','Target'],axis=1)
y = df_train['Target']

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# ML Models

In [ ]:
from sklearn.metrics import classification_report,accuracy_score
from sklearn import tree
from sklearn import ensemble
from sklearn import linear_model
from sklearn import svm
from sklearn import model_selection
from xgboost.sklearn import XGBClassifier
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,accuracy_score

MLA = [
    ensemble.RandomForestClassifier(n_jobs=-1, random_state = 0),
    XGBClassifier(n_jobs=-1),
]

MLA_columns = ['MLA_names', 'MLA_parameters', 'MLA_Train_Accuracy_Mean','MLA_Test_Accuracy_Mean']

def apply_models(X,y):
    MLA_compare = pd.DataFrame(columns = MLA_columns)

#     MLA_Predict = y_test.copy()
    row_index = 0
    for alg in tqdm(MLA):
        try:
            MLA_name = alg.__class__.__name__
            MLA_compare.loc[row_index, 'MLA_names'] = MLA_name
            MLA_compare.loc[row_index, 'MLA_parameters'] = str(alg.get_params())
            cv_results = model_selection.cross_validate(alg, X, y, cv=5, return_train_score = True)
            MLA_compare.loc[row_index, 'MLA_Train_Accuracy_Mean'] = cv_results["train_score"].mean()
            MLA_compare.loc[row_index, 'MLA_Test_Accuracy_Mean'] = cv_results['test_score'].mean()
            row_index += 1
            print(".", end="")
        except:
            print(type(alg).__name__)
    MLA_compare.sort_values(by = 'MLA_Test_Accuracy_Mean', ascending = False, inplace = True)
    return MLA_compare

In [ ]:
apply_models(X,y)

# Predict

In [ ]:
X_test = dfx_cleaned.drop(['uid'],axis=1)

X_test = scaler.transform(X_test)

In [ ]:
alg = XGBClassifier(n_jobs=-1)
alg.fit(X, y)
y_pred = alg.predict(X_test)

In [ ]:
CX_pred = pd.DataFrame({'uid':dfx_cleaned['uid'].tolist(),'Pred':y_pred})
CX_pred

# Fake

In [ ]:
import folium
# pip install folium
def draw(data):
    data.reset_index(drop=True,inplace=True)
    
    map_center = [data["lat"][0], data["lon"][0]]
    mymap = folium.Map(location=map_center, zoom_start=10)
    
    # Add start marker
    folium.Marker(location=[data["lat"].values[0], data["lon"].values[0]], icon=folium.Icon(color="green"), popup="Start").add_to(mymap)

    # Add end marker
    folium.Marker(location=[data["lat"].values[-1], data["lon"].values[-1]], icon=folium.Icon(color="red"), popup="End").add_to(mymap)

    
    for lat,long in zip(data['lat'],data['lon']):
        folium.CircleMarker([lat,long],radius=5, color='red',fill=True).add_to(mymap)
    display(mymap)

In [ ]:
temp = df1[df1['uid']==7]
temp

draw(temp)

In [ ]:
temp = df1[df1['uid']==5]
temp

draw(temp)

# Real

In [ ]:
temp = df2[df2['uid']==1]
temp

draw(temp)

In [ ]:
temp = df2[df2['uid']==2]
temp

draw(temp)

# Prediction

In [ ]:
CX_pred

In [ ]:
temp = dfx[dfx['uid']==118] # incorrect prediciton
temp

draw(temp)

In [ ]:
temp = dfx[dfx['uid']==418] # correct prediction
temp

draw(temp)

In [ ]:
# Q2
using folium we can plot the interacgtive view so as to check if its having issue or not

# Q3
- add distance between actual start and end of gps location
- Also, the beginning and end of the gps location as per scale on map

# Q4

- yes this data can be used for traffic analysis. first we need to find out the read gps traces and then overlap of their traces

# Q5

No, we need to perform feature engineering to create few features having enough potential to classified as issues/Not

# Q6

in the above machine learning models, we have performed prediciton of CX dataset

# Q7

- add distance between actual start and end of gps location
- Also, the beginning and end of the gps location as per scale on map
- 

# Future scope

In [ ]:
def get_features(train):
    train['longitude_diff'] = train.groupby('uid').lon.diff().fillna(0)
    train['latitude_diff'] = train.groupby('uid').lat.diff().fillna(0)
    train['elevation_diff'] = train.groupby('uid').ele.diff().fillna(0)

    train['UTMX'] = train.progress_apply(lambda x:utm.from_latlon(x['lat'], x['lon'])[0],axis=1)
    train['UTMY'] = train.progress_apply(lambda x:utm.from_latlon(x['lat'], x['lon'])[1],axis=1)

    # Function to calculate distance between two points
    distance = lambda x_dif, y_dif: np.sqrt(x_dif**2 + y_dif**2)

    # Differencing UTM coordinates
    train['UTMX_diff'] = train.groupby('uid').UTMX.diff().fillna(0)
    train['UTMY_diff'] = train.groupby('uid').UTMY.diff().fillna(0)


    # Calculate step distance
    train['distance'] = distance(train.UTMX_diff, train.UTMY_diff)

    # Interquartile and range function
    iqr = lambda x: np.percentile(x, 75) - np.percentile(x, 25)
    range = lambda x: np.max(x) - np.min(x)
    
    # Drop the last 1 column: label
    
    if 'Target' and 'row_number' in train:
        df_grouped = train.drop(['Target','row_number'],axis=1)
    else:
        df_grouped = train.drop(['row_number'],axis=1)

    # Calculate summary statistics
    df_grouped = df_grouped.groupby('uid').aggregate([np.mean, np.min, np.max, np.std, iqr, range])

    # Reduce multi-index
    df_grouped.columns = ['_'.join(col).strip() for col in df_grouped.columns.values]

    # Replace column name <lambda_0> to IQR and <lambda_1> to range 
    col_groupby2 = df_grouped.columns
    col_groupby2 = [w.replace('<lambda_0>', 'IQR') for w in col_groupby2]
    col_groupby2 = [w.replace('<lambda_1>', 'range') for w in col_groupby2]

    # Update names of columns
    df_grouped.columns = col_groupby2
    return df_grouped

In [ ]:
df_grouped